#Paquetes necesarios

In [ ]:
import cv2  
import math 

from ultralytics import YOLO

Modelos preentrenados, visualizando con las utilidades de ultralytics

In [ ]:
# Carga del modelo
#model = YOLO('yolo11n.pt') #Contenedores
#model = YOLO('yolo11n-seg.pt') #Máscaras
model = YOLO('yolo11n-pose.pt')  #Pose

#Para un vídeo 
filename = "TGC23_PdH_C0056cut.mp4"
results = model(filename, show=True)

cv2.destroyAllWindows()

Desde cámara, detección con yolo11, modelo nano. Visualización propia con OpenCV

In [ ]:
# Carga del modelo, descarga en disco si no está presente en la carpeta
model = YOLO('yolo11n.pt') #Contenedores

# Etiqueta de las distintas clases
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]


# Captura desde la webcam
vid = cv2.VideoCapture(0)
  
while(True):      
    # fotograma a fotograma
    ret, img = vid.read()
  
    # si hay imagen válida
    if ret:  
        # Detecta en la imagen
        results = model(img, stream=True)
        
        # Para cada detección
        for r in results:
            boxes = r.boxes

            for box in boxes:
                # Contenedor
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values
                
                # Confianza
                confidence = math.ceil((box.conf[0]*100))/100
                print("Confianza --->",confidence)

                # Clase
                cls = int(box.cls[0])
                print("Clase -->", classNames[cls])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255*2:
                    R = 255
                    G = 255
                    B = escala - 255*2
                else:
                    if escala >= 255:
                        R = 255
                        G = escala - 255
                        B = 0
                    else:
                        R = escala
                        G = 0
                        B = 0

                # Dibuja el contenedor y clase
                cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
                cv2.putText(img, classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

        # Muestra fotograma
        cv2.imshow('Vid', img)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

Seguimiento. Requiere instalar lap con pip install lap

In [ ]:
from collections import defaultdict
import numpy as np

# Carga del modelo, descarga en disco si no está presente en la carpeta
model = YOLO('yolo11n.pt') #Contenedores

# Etiqueta de las distintas clases
classNames = ["person", "bicycle", "car"]


# Captura desde la webcam
vid = cv2.VideoCapture(0)
track_history = defaultdict(lambda: [])
  
while(True):      
    # fotograma a fotograma
    ret, img = vid.read()
  
    # si hay imagen válida
    if ret:  
        # Seguimiento, con persistencia entre fotogramas
        results = model.track(img, persist=True, classes = [0,1,2])

        if 0:
            if results is not None:
                print(results[0])
                boxes = results[0].boxes.xywh.cpu()
                track_ids = results[0].boxes.id.int().cpu().tolist()
                annotated_frame = results[0].plot()
                for box, track_id in zip(boxes, track_ids):
                    x, y, w, h = box
                    track = track_history[track_id]
                    track.append((float(x), float(y)))
                    if len(track) > 30:
                        track.pop(0)
                    points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                    cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)
                cv2.imshow("YOLO11 Tracking", annotated_frame)
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
        

        
        # Para cada detección
        for r in results:
            boxes = r.boxes

            for box in boxes:
                # Contenedor
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

                #Etiqueta de seguimiento
                if box.id is not None:
                    track_id = str(int(box.id[0].tolist()))
                else:
                    track_id = ''
                
                # Confianza
                confidence = math.ceil((box.conf[0]*100))/100
                print("Confianza --->",confidence)

                # Clase
                cls = int(box.cls[0])
                print("Clase -->", classNames[cls])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255*2:
                    R = 255
                    G = 255
                    B = escala - 255*2
                else:
                    if escala >= 255:
                        R = 255
                        G = escala - 255
                        B = 0
                    else:
                        R = escala
                        G = 0
                        B = 0

                # Dibuja el contenedor y clase
                cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
                cv2.putText(img, track_id + ' ' + classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

        # Muestra fotograma
        cv2.imshow('Vid', img)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()



In [4]:
"""
PIPELINE GPU OPTIMIZADO - ANONIMIZACIÓN SIN OCR (MEJOR DETECCIÓN DE MATRÍCULAS)
Autor: Corazoncito de Melocotón 💛

Características:
- YOLO11n para detección general + YOLO especializado para matrículas
- Blur estable y centrado
- IDs visibles en vídeo
- Detección más robusta de matrículas (sin OCR)
- GPU optimizada, máximo rendimiento
"""

# ========================= CONFIG =========================
VIDEO_IN_PATH = "C0142.MP4"
VIDEO_OUT_PATH = "outputs/salida_anonimizada.mp4"
CSV_OUT_PATH = "outputs/detecciones.csv"

GENERAL_MODEL = "yolo11n.pt"
PLATE_MODEL = "best.pt"

CONF_THRESHOLD = 0.25
BLUR_INTENSITY = 61
MOVEMENT_THRESHOLD = 50
# ============================================================

import os, csv, cv2, torch
import numpy as np
from ultralytics import YOLO
from collections import defaultdict

# GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\n[INFO] Dispositivo: {device.upper()}")
if device == 'cuda':
    print(f"[INFO] GPU: {torch.cuda.get_device_name(0)}")

os.makedirs("outputs", exist_ok=True)

# Modelos
model_general = YOLO(GENERAL_MODEL).to(device)
model_plate = YOLO(PLATE_MODEL).to(device)

# Clases que nos interesan
classNames = ["person", "bicycle", "car", "motorbike", "bus", "truck"]
VEHICLE_CLASSES = {"car", "bus", "truck", "motorbike", "bicycle"}

# ====== FUNCIONES ======
def blur_region(img, x1, y1, x2, y2, intensity=BLUR_INTENSITY):
    """Aplica blur controlando límites"""
    h, w = img.shape[:2]
    x1, y1, x2, y2 = map(int, [max(0, x1), max(0, y1), min(w, x2), min(h, y2)])
    if x2 <= x1 or y2 <= y1:
        return img
    roi = img[y1:y2, x1:x2]
    k = intensity if (x2 - x1) > 30 else 15
    k = k if k % 2 == 1 else k + 1
    blurred = cv2.GaussianBlur(roi, (k, k), 0)
    img[y1:y2, x1:x2] = blurred
    return img

def smooth_coords(prev, new, alpha=0.5):
    """Suavizado exponencial de coordenadas"""
    if prev is None:
        return new
    return tuple(int(p * (1 - alpha) + n * alpha) for p, n in zip(prev, new))

# ====== VIDEO ======
cap = cv2.VideoCapture(VIDEO_IN_PATH)
width, height = int(cap.get(3)), int(cap.get(4))
fps = cap.get(5) or 25
total_frames = int(cap.get(7))

out = cv2.VideoWriter(VIDEO_OUT_PATH, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

# ====== CSV ======
csv_file = open(CSV_OUT_PATH, "w", newline="", encoding="utf-8")
writer = csv.writer(csv_file)
writer.writerow(["frame", "tipo_objeto", "confianza", "id_tracking",
                 "x1", "y1", "x2", "y2",
                 "matricula_detectada", "conf_matricula",
                 "mx1", "my1", "mx2", "my2"])

# ====== TRACKING ======
results_stream = model_general.track(
    source=VIDEO_IN_PATH,
    tracker="botsort.yaml",
    stream=True,
    device=device,
    classes=[0, 1, 2, 3, 5, 7],  # Solo personas, bicis, coches, motos, bus, camión
    verbose=False  # Desactiva los prints de YOLO
)

total_detections = defaultdict(set)
plate_tracker = {}
frame_id = 0

print(f"\n[INFO] Procesando {total_frames} frames...\n")

# ============================================================
# LOOP PRINCIPAL
# ============================================================
for res in results_stream:
    frame_id += 1
    frame = res.orig_img.copy()
    boxes = res.boxes

    for box in boxes:
        cls_id = int(box.cls)
        if cls_id >= len(classNames):
            continue
        label = classNames[cls_id]

        x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
        conf = float(box.conf)
        track_id = int(box.id) if box.id is not None else -1

        # --- Conteo acumulado ---
        if track_id != -1:
            total_detections[label].add(track_id)

        # ===== PERSONAS =====
        if label == "person":
            frame = blur_region(frame, x1, y1, x2, y2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, f"Person {track_id}", (x1, y1 - 8),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
            continue

        # ===== VEHÍCULOS =====
        if label in VEHICLE_CLASSES:
            mx1 = my1 = mx2 = my2 = -1
            plate_conf = 0.0
            plate_found = False

            crop = frame[y1:y2, x1:x2]
            if crop.size > 0:
                lp_res = model_plate.predict(crop, conf=CONF_THRESHOLD, device=device, verbose=False)
                if len(lp_res[0].boxes) > 0:
                    # --- Usa la de mayor confianza ---
                    best_box = max(lp_res[0].boxes, key=lambda b: b.conf)
                    lx1, ly1, lx2, ly2 = map(int, best_box.xyxy[0].cpu().numpy())
                    mx1, my1, mx2, my2 = x1 + lx1, y1 + ly1, x1 + lx2, y1 + ly2
                    plate_conf = float(best_box.conf)
                    plate_found = True

            # --- Suavizado de coordenadas ---
            if track_id != -1:
                prev = plate_tracker.get(track_id, {}).get("pos")
                new_coords = (mx1, my1, mx2, my2) if plate_found else prev
                smoothed = smooth_coords(prev, new_coords) if new_coords else None
                if smoothed:
                    mx1, my1, mx2, my2 = smoothed
                    plate_tracker[track_id] = {"pos": smoothed, "conf": plate_conf}

            # --- Blur de matrícula ---
            if track_id in plate_tracker and plate_tracker[track_id].get("pos"):
                mx1, my1, mx2, my2 = plate_tracker[track_id]["pos"]
                frame = blur_region(frame, mx1, my1, mx2, my2)
                cv2.rectangle(frame, (mx1, my1), (mx2, my2), (0, 255, 0), 2)

            # --- Caja del vehículo ---
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 128, 255), 2)
            cv2.putText(frame, f"{label} {track_id}", (x1, y1 - 8),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 128, 255), 2)

            writer.writerow([frame_id, label, round(conf, 3), track_id,
                             x1, y1, x2, y2, int(plate_found), round(plate_conf, 3),
                             mx1, my1, mx2, my2])

    out.write(frame)

    # Imprimir progreso cada 100 frames
    if frame_id % 100 == 0:
        print(f"[INFO] Procesados {frame_id}/{total_frames} frames ({frame_id*100//total_frames}%)")

cap.release()
out.release()
csv_file.close()

# ============================================================
# RESULTADOS
# ============================================================
print("\n========== RESULTADOS ==========")
for cls, ids in sorted(total_detections.items()):
    print(f"{cls}: {len(ids)} detectados en total")
print("================================")


[INFO] Dispositivo: CUDA
[INFO] GPU: NVIDIA GeForce RTX 4060 Laptop GPU

[INFO] Procesando 2832 frames...

[INFO] Procesados 100/2832 frames (3%)
[INFO] Procesados 200/2832 frames (7%)
[INFO] Procesados 300/2832 frames (10%)
[INFO] Procesados 400/2832 frames (14%)
[INFO] Procesados 500/2832 frames (17%)
[INFO] Procesados 600/2832 frames (21%)
[INFO] Procesados 700/2832 frames (24%)
[INFO] Procesados 800/2832 frames (28%)
[INFO] Procesados 900/2832 frames (31%)
[INFO] Procesados 1000/2832 frames (35%)
[INFO] Procesados 1100/2832 frames (38%)
[INFO] Procesados 1200/2832 frames (42%)
[INFO] Procesados 1300/2832 frames (45%)
[INFO] Procesados 1400/2832 frames (49%)
[INFO] Procesados 1500/2832 frames (52%)
[INFO] Procesados 1600/2832 frames (56%)
[INFO] Procesados 1700/2832 frames (60%)
[INFO] Procesados 1800/2832 frames (63%)
[INFO] Procesados 1900/2832 frames (67%)
[INFO] Procesados 2000/2832 frames (70%)
[INFO] Procesados 2100/2832 frames (74%)
[INFO] Procesados 2200/2832 frames (77%)
[

In [5]:
"""
PIPELINE GPU OPTIMIZADO - ANONIMIZACIÓN CON DETECCIÓN DUAL DE MATRÍCULAS
Autor: Corazoncito de Melocotón 💛

Características:
- YOLO11n para detección general + YOLO especializado para matrículas
- EasyOCR para detectar región de matrícula (sin leer texto)
- Blur estable y centrado
- IDs visibles en vídeo
- Detección más robusta combinando YOLO + EasyOCR
- GPU optimizada, máximo rendimiento
"""

# ========================= CONFIG =========================
VIDEO_IN_PATH = "C0142.MP4"
VIDEO_OUT_PATH = "outputs/salida_anonimizada.mp4"
CSV_OUT_PATH = "outputs/detecciones.csv"

GENERAL_MODEL = "yolo11n.pt"
PLATE_MODEL = "best.pt"

CONF_THRESHOLD = 0.25
BLUR_INTENSITY = 61
USE_EASYOCR = True  # Activar/desactivar EasyOCR como detector adicional
# ============================================================

import os, csv, cv2, torch
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
import easyocr

# GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\n[INFO] Dispositivo: {device.upper()}")
if device == 'cuda':
    print(f"[INFO] GPU: {torch.cuda.get_device_name(0)}")

os.makedirs("outputs", exist_ok=True)

# Modelos
model_general = YOLO(GENERAL_MODEL).to(device)
model_plate = YOLO(PLATE_MODEL).to(device)

# EasyOCR (solo para detectar región, no leer)
if USE_EASYOCR:
    print("[INFO] Inicializando EasyOCR...")
    reader = easyocr.Reader(['en'], gpu=(device == 'cuda'), verbose=False)
    print("[INFO] EasyOCR listo")

# Clases que nos interesan
classNames = ["person", "bicycle", "car", "motorbike", "bus", "truck"]
VEHICLE_CLASSES = {"car", "bus", "truck", "motorbike", "bicycle"}

# ====== FUNCIONES ======
def blur_region(img, x1, y1, x2, y2, intensity=BLUR_INTENSITY):
    """Aplica blur controlando límites"""
    h, w = img.shape[:2]
    x1, y1, x2, y2 = map(int, [max(0, x1), max(0, y1), min(w, x2), min(h, y2)])
    if x2 <= x1 or y2 <= y1:
        return img
    roi = img[y1:y2, x1:x2]
    k = intensity if (x2 - x1) > 30 else 15
    k = k if k % 2 == 1 else k + 1
    blurred = cv2.GaussianBlur(roi, (k, k), 0)
    img[y1:y2, x1:x2] = blurred
    return img

def smooth_coords(prev, new, alpha=0.5):
    """Suavizado exponencial de coordenadas"""
    if prev is None:
        return new
    return tuple(int(p * (1 - alpha) + n * alpha) for p, n in zip(prev, new))

def detect_plate_with_ocr(crop):
    """Detecta región de matrícula usando EasyOCR (sin leer)"""
    try:
        results = reader.readtext(crop, paragraph=False)
        if results:
            # Buscar el texto más parecido a una matrícula (alfanumérico)
            for (bbox, text, conf) in results:
                # Solo nos interesan detecciones con confianza razonable
                if conf > 0.3 and any(c.isalnum() for c in text):
                    # bbox es [[x1,y1], [x2,y1], [x2,y2], [x1,y2]]
                    pts = np.array(bbox, dtype=np.int32)
                    x1 = int(pts[:, 0].min())
                    y1 = int(pts[:, 1].min())
                    x2 = int(pts[:, 0].max())
                    y2 = int(pts[:, 1].max())
                    return (x1, y1, x2, y2, conf)
    except:
        pass
    return None

def merge_detections(yolo_box, ocr_box):
    """Combina detecciones de YOLO y OCR para obtener mejor región"""
    if yolo_box is None and ocr_box is None:
        return None
    if yolo_box is None:
        return ocr_box
    if ocr_box is None:
        return yolo_box
    
    # Usar la detección con mayor confianza o fusionar
    yx1, yy1, yx2, yy2, yconf = yolo_box
    ox1, oy1, ox2, oy2, oconf = ocr_box
    
    # Si las cajas se solapan, usar la de mayor confianza
    if yconf > oconf:
        return yolo_box
    else:
        return ocr_box

# ====== VIDEO ======
cap = cv2.VideoCapture(VIDEO_IN_PATH)
width, height = int(cap.get(3)), int(cap.get(4))
fps = cap.get(5) or 25
total_frames = int(cap.get(7))

out = cv2.VideoWriter(VIDEO_OUT_PATH, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

# ====== CSV ======
csv_file = open(CSV_OUT_PATH, "w", newline="", encoding="utf-8")
writer = csv.writer(csv_file)
writer.writerow(["frame", "tipo_objeto", "confianza", "id_tracking",
                 "x1", "y1", "x2", "y2",
                 "matricula_detectada", "conf_matricula", "metodo_deteccion",
                 "mx1", "my1", "mx2", "my2"])

# ====== TRACKING ======
results_stream = model_general.track(
    source=VIDEO_IN_PATH,
    tracker="botsort.yaml",
    stream=True,
    device=device,
    classes=[0, 1, 2, 3, 5, 7],  # Solo personas, bicis, coches, motos, bus, camión
    verbose=False  # Desactiva los prints de YOLO
)

total_detections = defaultdict(set)
plate_tracker = {}
frame_id = 0

print(f"\n[INFO] Procesando {total_frames} frames...\n")

# ============================================================
# LOOP PRINCIPAL
# ============================================================
for res in results_stream:
    frame_id += 1
    frame = res.orig_img.copy()
    boxes = res.boxes

    for box in boxes:
        cls_id = int(box.cls)
        if cls_id >= len(classNames):
            continue
        label = classNames[cls_id]

        x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
        conf = float(box.conf)
        track_id = int(box.id) if box.id is not None else -1

        # --- Conteo acumulado ---
        if track_id != -1:
            total_detections[label].add(track_id)

        # ===== PERSONAS =====
        if label == "person":
            frame = blur_region(frame, x1, y1, x2, y2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, f"Person {track_id}", (x1, y1 - 8),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
            continue

        # ===== VEHÍCULOS =====
        if label in VEHICLE_CLASSES:
            mx1 = my1 = mx2 = my2 = -1
            plate_conf = 0.0
            plate_found = False
            detection_method = "none"

            crop = frame[y1:y2, x1:x2]
            if crop.size > 0:
                yolo_detection = None
                ocr_detection = None
                
                # Detección con YOLO
                lp_res = model_plate.predict(crop, conf=CONF_THRESHOLD, device=device, verbose=False)
                if len(lp_res[0].boxes) > 0:
                    best_box = max(lp_res[0].boxes, key=lambda b: b.conf)
                    lx1, ly1, lx2, ly2 = map(int, best_box.xyxy[0].cpu().numpy())
                    yolo_conf = float(best_box.conf)
                    yolo_detection = (lx1, ly1, lx2, ly2, yolo_conf)
                    detection_method = "yolo"
                
                # Detección con EasyOCR (solo cada N frames para no ralentizar)
                if USE_EASYOCR and frame_id % 5 == 0:  # Solo cada 5 frames
                    ocr_result = detect_plate_with_ocr(crop)
                    if ocr_result:
                        ocr_detection = ocr_result
                        if yolo_detection is None:
                            detection_method = "ocr"
                        else:
                            detection_method = "yolo+ocr"
                
                # Combinar detecciones
                final_detection = merge_detections(yolo_detection, ocr_detection)
                
                if final_detection:
                    lx1, ly1, lx2, ly2, plate_conf = final_detection
                    mx1, my1, mx2, my2 = x1 + lx1, y1 + ly1, x1 + lx2, y1 + ly2
                    plate_found = True

            # --- Suavizado de coordenadas ---
            if track_id != -1:
                prev = plate_tracker.get(track_id, {}).get("pos")
                new_coords = (mx1, my1, mx2, my2) if plate_found else prev
                smoothed = smooth_coords(prev, new_coords) if new_coords else None
                if smoothed:
                    mx1, my1, mx2, my2 = smoothed
                    plate_tracker[track_id] = {"pos": smoothed, "conf": plate_conf}

            # --- Blur de matrícula ---
            if track_id in plate_tracker and plate_tracker[track_id].get("pos"):
                mx1, my1, mx2, my2 = plate_tracker[track_id]["pos"]
                frame = blur_region(frame, mx1, my1, mx2, my2)
                cv2.rectangle(frame, (mx1, my1), (mx2, my2), (0, 255, 0), 2)

            # --- Caja del vehículo ---
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 128, 255), 2)
            cv2.putText(frame, f"{label} {track_id}", (x1, y1 - 8),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 128, 255), 2)

            writer.writerow([frame_id, label, round(conf, 3), track_id,
                             x1, y1, x2, y2, int(plate_found), round(plate_conf, 3),
                             detection_method, mx1, my1, mx2, my2])

    out.write(frame)

    # Imprimir progreso cada 100 frames
    if frame_id % 100 == 0:
        print(f"[INFO] Procesados {frame_id}/{total_frames} frames ({frame_id*100//total_frames}%)")

cap.release()
out.release()
csv_file.close()

# ============================================================
# RESULTADOS
# ============================================================
print("\n========== RESULTADOS ==========")
for cls, ids in sorted(total_detections.items()):
    print(f"{cls}: {len(ids)} detectados en total")
print("================================")


[INFO] Dispositivo: CUDA
[INFO] GPU: NVIDIA GeForce RTX 4060 Laptop GPU
[INFO] Inicializando EasyOCR...
[INFO] EasyOCR listo

[INFO] Procesando 2832 frames...

[INFO] Procesados 100/2832 frames (3%)
[INFO] Procesados 200/2832 frames (7%)
[INFO] Procesados 300/2832 frames (10%)
[INFO] Procesados 400/2832 frames (14%)
[INFO] Procesados 500/2832 frames (17%)
[INFO] Procesados 600/2832 frames (21%)
[INFO] Procesados 700/2832 frames (24%)
[INFO] Procesados 800/2832 frames (28%)
[INFO] Procesados 900/2832 frames (31%)
[INFO] Procesados 1000/2832 frames (35%)
[INFO] Procesados 1100/2832 frames (38%)
[INFO] Procesados 1200/2832 frames (42%)
[INFO] Procesados 1300/2832 frames (45%)
[INFO] Procesados 1400/2832 frames (49%)
[INFO] Procesados 1500/2832 frames (52%)
[INFO] Procesados 1600/2832 frames (56%)
[INFO] Procesados 1700/2832 frames (60%)
[INFO] Procesados 1800/2832 frames (63%)
[INFO] Procesados 1900/2832 frames (67%)
[INFO] Procesados 2000/2832 frames (70%)
[INFO] Procesados 2100/2832 fr

Intregración con seguimiento (tracking)
!!!!!!!!!Nota: he tenido que bajar a la versión de python 3.9.5 e instalar lap con pip install lap

In [ ]:
# Carga del modelo
model = YOLO('yolo11n.pt') #Contenedores
#model = YOLO('yolov11n-seg.pt') #Máscaras
#model = YOLO('yolo11n-pose.pt')  #Pose

#Para un vídeo 
filename = "TGC23_PdH_C0056cut.mp4"
results = model.track(source=filename, show=True)  # BoT-SORT tracker (por defecto)
#results = model.track(source=filename, show=True, tracker="bytetrack.yaml")  # ByteTrack tracker

cv2.destroyAllWindows()